In [5]:
# importar librerías
import sqlite3  # Módulo para trabajar con bases de datos SQLite3
import pandas as pd  # Módulo para trabajar con estructuras de datos tipo DataFrame
from ipywidgets import interact  # Módulo para crear widgets interactivos

# conectarse a la base de datos
conn = sqlite3.connect("data/db_movies")
cur = conn.cursor()  # Crear un cursor para ejecutar sentencias SQL

# consultamos las tablas de la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('ratings',), ('movies',), ('movies_clean',), ('full_table',), ('data_clean',)]


In [6]:
# df quitando los title duplicados para utilizar en los modelos
movies = pd.read_sql_query("SELECT * FROM data_clean", conn)
movies = movies.drop_duplicates(subset=["title"])
movies = movies.reset_index(drop=True)

# ver tabla de data_clean
df = pd.read_sql_query("SELECT * FROM data_clean", conn)
df.head(5)

,movieId,title,year,userId,rating,timestamp,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995,1,4.0,964982703,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),1995,5,4.0,847434962,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),1995,7,4.5,1106635946,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),1995,15,2.5,1510577970,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),1995,17,4.5,1305696483,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## FILTRO BASADO EN POPULARIDAD

### TOP 10 PELICULAS MEJOR CALIFICADAS

In [5]:
##### recomendaciones basado en popularidad ######
mej_cal = pd.read_sql(
    """select title, 
            avg(rating) as avg_rat,
            count(*) as cal_num
            from data_clean
            group by title
            order by avg_rat desc
            limit 10
            """,
    conn,
)
# Redondear 1 decimal a avg_rat
mej_cal["avg_rat"] = mej_cal["avg_rat"].round(1)
mej_cal

,title,avg_rat,cal_num
0,"Shawshank Redemption, The (1994)",4.4,302
1,Lawrence of Arabia (1962),4.3,44
2,"Godfather, The (1972)",4.3,182
3,Dr. Strangelove or: How I Learned to Stop Worr...,4.3,94
4,Patton (1970),4.3,31
5,Cool Hand Luke (1967),4.3,55
6,Chinatown (1974),4.3,57
7,Rear Window (1954),4.3,84
8,Fight Club (1999),4.3,210
9,"Princess Bride, The (1987)",4.3,136


### TOP 10 PELÍCULAS MÁS CALIFICADAS

In [6]:
##### recomendaciones basado en popularidad ######
mej_cal = pd.read_sql(
    """select title, 
            avg(rating) as avg_rat,
            count(*) as cal_num
            from data_clean
            group by title
            order by cal_num desc
            limit 10
            """,
    conn,
)
# Redondear 1 decimal a avg_rat
mej_cal["avg_rat"] = mej_cal["avg_rat"].round(1)
mej_cal

,title,avg_rat,cal_num
0,Forrest Gump (1994),4.2,315
1,"Shawshank Redemption, The (1994)",4.4,302
2,Pulp Fiction (1994),4.2,294
3,"Matrix, The (1999)",4.2,267
4,"Silence of the Lambs, The (1991)",4.2,263
5,Star Wars: Episode IV - A New Hope (1977),4.2,241
6,Jurassic Park (1993),3.7,229
7,Braveheart (1995),4.0,225
8,Terminator 2: Judgment Day (1991),4.0,217
9,Fight Club (1999),4.3,210


### TOP 10 PELÍCULAS MEJOR CALIFICADAS POR AÑO

In [9]:
# Función pidiendo el año
def BestMovieForYear(year=list(movies["year"].sort_values().unique()), top = 10):
    cal_year = pd.read_sql(
        """select year, title, 
            avg(rating) as avg_rat,
            count(*) as count_rat
            from data_clean
            group by  year, title
            order by year desc, avg_rat desc
            """,
        conn,
    )

    # Aproximar a 2 decimales
    cal_year = cal_year.round(1)

    # Se escogen de la tabla anterior 10 películas por año con mejor calificación pero mostrando todos los años
    cal_year = cal_year.groupby("year").head(top)

    return cal_year[cal_year["year"] == year]


print(interact(BestMovieForYear))

interactive(children=(Dropdown(description='year', options=('1937', '1939', '1940', '1941', '1942', '1946', '1…

<function BestMovieForYear at 0x00000231B3389DA0>


### TOP 10 PELICULAS CON MÁS CALIFICACIONES POR AÑO


In [10]:
def BestRankedforYears(year=list(movies["year"].sort_values().unique()), top = 10):
    # Buscar el índice de la película
    cal_year = pd.read_sql(
        """select year, title, 
            avg(rating) as avg_rat,
            count(*) as count_rat
            from data_clean
            group by  year, title
            order by year desc, count_rat desc
            """,
        conn,
    )

    # Aproximar a 2 decimales
    cal_year = cal_year.round(1)

    # Escoger de la tabla anterior 10 películas por año 
    cal_year = cal_year.groupby("year").head(top)
    cal_year = cal_year[cal_year["year"] == year]

    return cal_year

print(interact(BestRankedforYears))

interactive(children=(Dropdown(description='year', options=('1937', '1939', '1940', '1941', '1942', '1946', '1…

<function BestRankedforYears at 0x00000231B188FF60>


### TOP 1 PELÍCULA MEJOR CALIFICADA POR GÉNERO

In [11]:
# Mejores películas por género
# (nota: Film noir y Sci fi nos da problemas)

lista = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    # "Film-Noir",
    "Horror",
    "IMAX",
    "Musical",
    "Mystery",
    "Romance",
    # "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for i in lista:
    cal_action = pd.read_sql(
        f"""select title, 
                avg(rating) as avg_rat,
                count(*) as count_rat
                from data_clean
                where {i} = 1
                group by  title
                order by avg_rat desc limit 1
                """,
        conn,
    )

    best_avg = cal_action["avg_rat"].tolist()
    best_genres = cal_action["title"].tolist()
    best_count = cal_action["count_rat"].tolist()

    print(
        f"La mejor pelicula de {i} es: {best_genres[0]} con {best_count[0]} votos y {round(best_avg[0],1)} estrellas "
    )

La mejor pelicula de Action es: Fight Club (1999) con 210 votos y 4.3 estrellas 
La mejor pelicula de Adventure es: Lawrence of Arabia (1962) con 44 votos y 4.3 estrellas 
La mejor pelicula de Animation es: Spirited Away (Sen to Chihiro no kamikakushi) (2001) con 86 votos y 4.1 estrellas 
La mejor pelicula de Children es: Toy Story 3 (2010) con 53 votos y 4.1 estrellas 
La mejor pelicula de Comedy es: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) con 94 votos y 4.3 estrellas 
La mejor pelicula de Crime es: Shawshank Redemption, The (1994) con 302 votos y 4.4 estrellas 
La mejor pelicula de Documentary es: Roger & Me (1989) con 30 votos y 3.9 estrellas 
La mejor pelicula de Drama es: Shawshank Redemption, The (1994) con 302 votos y 4.4 estrellas 
La mejor pelicula de Fantasy es: Princess Bride, The (1987) con 136 votos y 4.3 estrellas 
La mejor pelicula de Horror es: Silence of the Lambs, The (1991) con 263 votos y 4.2 estrellas 
La mejor pelicula de IMAX e

### TOP 1 PELÍCULA CON MÁS CALIFICACIONES POR GÉNERO

In [12]:
# Mejores películas por género
# (nota: Film noir y Sci fi nos da problemas)

lista = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    # "Film-Noir",
    "Horror",
    "IMAX",
    "Musical",
    "Mystery",
    "Romance",
    # "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for i in lista:
    cal_action = pd.read_sql(
        f"""select title, 
                avg(rating) as avg_rat,
                count(*) as count_rat
                from data_clean
                where {i} = 1
                group by  title
                order by count_rat desc limit 1
                """,
        conn,
    )
    best_avg = cal_action["avg_rat"].tolist()
    best_count = cal_action["count_rat"].tolist()
    best_genres = cal_action["title"].tolist()

    print(
        f"La mejor pelicula de {i} es: {best_genres[0]} con {best_count[0]} votos y {round(best_avg[0],1)} estrellas "
    )

La mejor pelicula de Action es: Matrix, The (1999) con 267 votos y 4.2 estrellas 
La mejor pelicula de Adventure es: Star Wars: Episode IV - A New Hope (1977) con 241 votos y 4.2 estrellas 
La mejor pelicula de Animation es: Toy Story (1995) con 208 votos y 3.9 estrellas 
La mejor pelicula de Children es: Toy Story (1995) con 208 votos y 3.9 estrellas 
La mejor pelicula de Comedy es: Forrest Gump (1994) con 315 votos y 4.2 estrellas 
La mejor pelicula de Crime es: Shawshank Redemption, The (1994) con 302 votos y 4.4 estrellas 
La mejor pelicula de Documentary es: Bowling for Columbine (2002) con 58 votos y 3.8 estrellas 
La mejor pelicula de Drama es: Forrest Gump (1994) con 315 votos y 4.2 estrellas 
La mejor pelicula de Fantasy es: Toy Story (1995) con 208 votos y 3.9 estrellas 
La mejor pelicula de Horror es: Silence of the Lambs, The (1991) con 263 votos y 4.2 estrellas 
La mejor pelicula de IMAX es: Apollo 13 (1995) con 190 votos y 3.8 estrellas 
La mejor pelicula de Musical es: A

## FILTRO BASADO EN UN SOLO PRODUCTO (MANUAL)

In [20]:
from sklearn.preprocessing import MinMaxScaler

# Cambiar el tipo de dato de la columna year
movies["year"] = movies.year.astype("int64")

##### escalar para que año esté en el mismo rango ###
sc = MinMaxScaler()
movies[["year"]] = sc.fit_transform(movies[["year"]])

## eliminar filas que no se van a utilizar ###
movies_dum = movies.drop(columns=["movieId", "title", "userId", "rating", "timestamp"])
movies = movies.drop(columns=["userId", "rating", "timestamp"])

#### No es necesario convertir a dummies porque no se tienen más caracteristicas
movies_dum.head(5)

,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.734177,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0.734177,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.734177,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0.734177,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.734177,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [22]:
from ipywidgets import interact  # Módulo para crear widgets interactivos


def FilterOneItemManual(movie=list(movies["title"].sort_values()), top=10):
    # Buscar el índice de la película
    ind_movie = movies[movies["title"] == movie].index.values.astype(int)[0]
    # Calcular la correlación
    similar_movies = movies_dum.corrwith(movies_dum.iloc[ind_movie, :], axis=1)
    # Ordenar de mayor a menor
    similar_movies = similar_movies.sort_values(ascending=False)
    # Tomar las indicadas quitando la primera (que es la misma película)
    top_similar_movies = similar_movies.to_frame(name="correlación").iloc[1:top,]
    # Agregar el título
    top_similar_movies["title"] = movies["title"]

    return top_similar_movies


print(interact(FilterOneItemManual))

interactive(children=(Dropdown(description='movie', options=('(500) Days of Summer (2009)', '10 Things I Hate …

<function FilterOneItemManual at 0x00000231B338AC00>


## FILTRO BASADO EN UN SOLO PRODUCTO (KNN)

In [23]:
from sklearn import neighbors  ### basado en contenido un solo producto consumido


def FilterOneItemKNN(movie_name=list(movies["title"].sort_values()), top=10):
    # Instanciar y entrenar el modelo
    model = neighbors.NearestNeighbors(n_neighbors=top, metric="cityblock")
    model.fit(movies_dum)
    # Calcular la distancia y los índices de los vecinos
    dist, idlist = model.kneighbors(movies_dum)

    # Crear un DataFrame con las distancias y los índices de los vecinos
    distancias = pd.DataFrame(dist)
    id_list = pd.DataFrame(idlist)

    # Buscar el índice de la película
    movie_list_name = []
    movie_id = movies[movies["title"] == movie_name].index
    # Tomar las indicadas quitando la primera (que es la misma película)
    movie_id = movie_id[0]
    # For para buscar los nombres de las películas
    for newid in idlist[movie_id]:
        movie_list_name.append(movies.loc[newid].title)
    return movie_list_name


print(interact(FilterOneItemKNN))

interactive(children=(Dropdown(description='movie_name', options=('(500) Days of Summer (2009)', '10 Things I …

<function FilterOneItemKNN at 0x00000231CC2C9B20>


## FILTRO BASADO EN TODO EL CONTENIDO POR USUARIO (KNN)

In [25]:
# seleccionar usuario para recomendaciones ####
usuarios = pd.read_sql("select distinct (UserId) as UserId from data_clean", conn)

def FilterAllItemsXUserKNN(user_id=list(usuarios["UserId"].sort_values()), top=10):
    # seleccionar solo los ratings del usuario seleccionado
    ratings = pd.read_sql(f"select *from data_clean where {user_id} = UserId", conn)

    # convertir ratings del usuario a array
    l_movies_r = ratings["movieId"].to_numpy()

    # agregar la columna de movieId y titulo de la pelicula a dummie para filtrar y mostrar nombre
    movies_dum[["movieId", "title"]] = movies[["movieId", "title"]]

    # filtrar solo las peliculas que el usuario ha calificado
    movies_r = movies_dum[movies_dum["movieId"].isin(l_movies_r)]

    # eliminar la columna de movieId y titulo
    movies_r = movies_r.drop(columns=["movieId", "title"])
    # para usar group by y que quede en formato pandas tabla de centroide
    movies_r["indice"] = 1
    # centroide o perfil del usuario
    centroide = movies_r.groupby("indice").mean()

    # filtrar películas no calificadas
    movies_nr = movies_dum[~movies_dum["movieId"].isin(l_movies_r)]
    # eliminar nombre y movieId
    movies_nr = movies_nr.drop(columns=["movieId", "title"])

    # entrenar modelo
    model = neighbors.NearestNeighbors(n_neighbors=top, metric="cosine")
    model.fit(movies_nr)
    # Calcular la distancia y los índices de los vecinos
    dist, idlist = model.kneighbors(centroide)
    # queda en un array anidado, para sacarlo
    ids = idlist[0]
    # sacar el nombre de las peliculas recomendadas
    recomend_b = movies.loc[ids][["title", "movieId"]]
    leidos = movies[movies["movieId"].isin(l_movies_r)][["title", "movieId"]]

    return recomend_b


print(interact(FilterAllItemsXUserKNN))

interactive(children=(Dropdown(description='user_id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …

<function FilterAllItemsXUserKNN at 0x00000231CC8CFCE0>


## FILTRO COLABORATIVO

### Modelos

In [7]:
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

### datos originales en pandas
## knn solo sirve para calificaciones explicitas

# Cargar datos de data_clean con solo las columnas necesarias (userId, movieId, rating)
ratings = pd.read_sql("select userId, movieId, rating from data_clean", conn)


####los datos deben ser Leidos en un formato especial para surprise
reader = Reader(rating_scale=(1, 5))  ### la escala de la calificación
###las columnas deben estar en orden estándar: user item rating
data = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], reader)


#####Existen varios modelos
models = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline()]
results = {}

###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación


#### función para probar varios modelos ##########

for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE", "RMSE"], cv=5, n_jobs=-1)

    result = (
        pd.DataFrame.from_dict(CV_scores)
        .mean(axis=0)
        .rename({"test_mae": "MAE", "test_rmse": "RMSE"})
    )
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by="RMSE")

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.631585,0.828126,0.344125,2.253295
knns.KNNWithZScore,0.636569,0.838372,0.357083,2.107991
knns.KNNWithMeans,0.641057,0.840087,0.292112,1.849719
knns.KNNBasic,0.674551,0.883805,0.268361,1.532916


### GridSearch

In [8]:
###################se escoge el mejor knn withmeans#########################
param_grid = {
    "min_k" : [10], # min. de vecinos para la similitud
    "sim_options": {  # opciones de similitud
        "name": ["msd", "cosine"],  # medidas de distancia
        "min_support": [10],  
        "user_based": [False, True],  # si es basado en usuario o basado en ítem
    }
}

### se afina si es basado en usuario o basado en ítem

gridsearch = GridSearchCV(
    KNNBaseline,
    param_grid,
    measures=["rmse"],
    cv=5,
    n_jobs=-1,
)

gridsearch.fit(data)


print(gridsearch.best_params["rmse"])
print(gridsearch.best_score["rmse"])
gs_model = gridsearch.best_estimator["rmse"]  ### mejor estimador de gridsearch

{'min_k': 10, 'sim_options': {'name': 'msd', 'min_support': 10, 'user_based': False}}
0.8179420103869663


### Mejor modelo

In [9]:
################# Entrenar con todos los datos y Realizar predicciones con el modelo afinado

trainset = (
    data.build_full_trainset()
)  ### esta función convierte todos los datos en entrenamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model = gs_model.fit(
    trainset
)  ## se reentrena sobre todos los datos posibles (sin dividir)

predset = (
    trainset.build_anti_testset()
)  ### crea una tabla con todos los usuarios y los libros que no han leido
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)

predictions = gs_model.test(
    predset
)  ### función muy pesada, hace las predicciones de rating para todos los libros que no hay leido un usuario
### la funcion test recibe un test set constriuido con build_test method, o el que genera crosvalidate

predictions_df = pd.DataFrame(
    predictions
)  ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df["r_ui"].unique()  ### promedio de ratings
predictions_df.sort_values(by="est", ascending=False)

# Ver la metrica de error
from surprise import accuracy

print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.5649
0.5648618007859041


Tenemos un RMSE de 0.5649 significa que en promedio nuestro modelo se equivoca en 0.5649 estrellas a la hora de calificar, es decir si realmente es 3.5 puede estimar 3.0 o 4.0.

In [12]:
predictions_df.sort_values(by="est", ascending=False)

,uid,iid,r_ui,est,details
354329,543,1272,3.66305,5.0,"{'actual_k': 18, 'was_impossible': False}"
395440,154,1219,3.66305,5.0,"{'actual_k': 15, 'was_impossible': False}"
69522,276,1222,3.66305,5.0,"{'actual_k': 29, 'was_impossible': False}"
315657,371,318,3.66305,5.0,"{'actual_k': 30, 'was_impossible': False}"
395433,154,1206,3.66305,5.0,"{'actual_k': 22, 'was_impossible': False}"
...,...,...,...,...,...
245586,311,1562,3.66305,1.0,"{'actual_k': 19, 'was_impossible': False}"
271879,3,30793,3.66305,1.0,"{'actual_k': 13, 'was_impossible': False}"
243041,287,762,3.66305,1.0,"{'actual_k': 40, 'was_impossible': False}"
271735,3,4025,3.66305,1.0,"{'actual_k': 11, 'was_impossible': False}"


En resumen, la salida (fila 4) muestra que, para el usuario con el identificador 371 y la película con el identificador 318, el sistema ha estimado una calificación de 5.0, mientras que la calificación real dada por el usuario fue de 3.66305 (vemos una diferencia grande, acá el desempeño del modelo juega un papel  para que la diferencia sea minima). 

El sistema consideró 30 (usuarios o items) (actual_k) que calificaron de una manera similar la película para hacer esta predicción, con esto se obtiene una tabla de predicciones para todos los usuarios con todas las predicciones de películas calificadas y no calificadas

### Interactivo

In [13]:
from ipywidgets import interact  # Módulo para crear widgets interactivos

# seleccionar usuario para recomendaciones ####
usuarios = pd.read_sql("select distinct (UserId) as UserId from data_clean", conn)


##### funcion para recomendar las 10 peliculas con mejores predicciones y llevar base de datos para consultar resto de información
def ColaborativeUserRecommender(
    user_id=list(usuarios["UserId"].sort_values()), n_recomend=10
):
    # seleccionar solo los ratings del usuario seleccionado
    ratings = pd.read_sql(f"select *from data_clean where {user_id} = UserId", conn)

    predictions_userID = (
        predictions_df[predictions_df["uid"] == user_id]
        .sort_values(by="est", ascending=False)
        .head(n_recomend)
    )

    recomendados = predictions_userID[["iid", "est", "details"]]
    # Si desea enviar a una base de datos
    # recomendados.to_sql('reco',conn,if_exists="replace")
    """
    recomendados = pd.read_sql(
        select a.*, b.book_title 
                             from reco a left join books_final b
                             on a.iid=b.isbn ,
        conn,
    )
    """
    # De la tabla movies buscar el título de las películas recomendadas
    recomendados = pd.merge(
        recomendados, movies[["movieId", "title"]], left_on="iid", right_on="movieId"
    )

    return recomendados


print(interact(ColaborativeUserRecommender))

interactive(children=(Dropdown(description='user_id', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …

<function ColaborativeUserRecommender at 0x0000026A2147FF60>


* Usuario 56: El objetivo es recomendar películas al Usuario 56.

* Búsqueda de películas similares a las que le gustaron: El sistema comienza identificando las películas que el Usuario 56 ya ha calificado positivamente. Estas películas sirven como puntos de partida para las recomendaciones.

* Identificación de usuarios similares: A continuación, el sistema busca un grupo de usuarios que también hayan calificado positivamente las mismas películas que le gustaron al Usuario 56. Esto implica encontrar usuarios que tengan un historial de calificaciones similar en términos de las películas que les gustaron.

* Recopilación de películas no calificadas por el Usuario 56: Una vez que se ha identificado un grupo de usuarios con perfiles similares al Usuario 56, el sistema recopila las películas que estos usuarios han calificado positivamente pero que el Usuario 56 aún no ha calificado.

* Ordenamiento y recomendación: Estas películas se ordenan en función de su popularidad o de algún otro criterio relevante, y luego se recomiendan las 10 películas mejor clasificadas al Usuario 56.

Este proceso aprovecha la idea de que los usuarios que tienen gustos similares en el pasado (basados en sus calificaciones) tienden a tener preferencias similares en el futuro. Por lo tanto, al identificar películas populares entre usuarios con gustos similares y que al Usuario 56 le gustaron las películas relacionadas, se generan recomendaciones personalizadas.

Si el filtro colaborativo hubiera sido por usuario el filtro sería de la siguiente manera

* Usuario 56: El objetivo sigue siendo recomendar películas al Usuario 56.

* Búsqueda de usuarios similares: En lugar de comenzar buscando películas que le gustaron al Usuario 56, el sistema buscará usuarios que sean similares al Usuario 56 en términos de sus historiales de calificación. Esto implica encontrar otros usuarios cuyas calificaciones pasadas sean similares a las del Usuario 56.

* Identificación de películas recomendadas por usuarios similares: Una vez que se ha identificado un grupo de usuarios con perfiles de calificación similares al Usuario 56, el sistema examina las películas que estos usuarios han calificado positivamente y que el Usuario 56 aún no ha visto o calificado.

* Ordenamiento y recomendación: Estas películas se pueden ordenar según algún criterio, como la popularidad entre los usuarios similares o la afinidad histórica del Usuario 56 con esas películas. Luego, se recomiendan las 10 películas mejor clasificadas al Usuario 56.


Las principales diferencias con respecto al filtrado colaborativo por ítem son:

* En el filtrado colaborativo por usuario, se buscan usuarios similares en lugar de ítems similares. Se parte de la idea de que usuarios con historiales de calificación similares tienen gustos similares en general.

* Las películas recomendadas se basan en las calificaciones y preferencias de usuarios similares, en lugar de buscar películas similares a las que le gustaron al Usuario 56.

En resumen, el filtrado colaborativo por usuario se centra en encontrar usuarios similares y recomendar películas que esos usuarios similares hayan disfrutado y que el Usuario 56 aún no haya visto, basándose en la premisa de que usuarios similares tienen gustos similares.